## Machine Learning item x item
### def **recomendacion_juego( *`user`* )**:
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos para dicho usuario.


___

In [3]:
#install surprise
#%pip install surprise

In [4]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import pyarrow as pa
import pyarrow.parquet as pq
import os

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD
import pickle

In [5]:
# Se cargan los archivos con los que se trabajará:
df_steam_games = pd.read_parquet('../0 Dataset/1.1_steam_games_LISTO.parquet')
df_user_reviews = pd.read_parquet('../0 Dataset/1.3_user_review_sentiment.parquet')
df_user_items = pd.read_parquet('../0 Dataset/1.2_users_items_LISTO.parquet')

Según la información proporcionada en la documentación del modelo 📄, se necesitan los siguientes parámetros para su funcionamiento:

* Usuario 👤
* Ítem 🎮
* Calificación ⭐

In [6]:
# Filtramos las variables en el dataframe
df_RS = df_user_reviews[['user_id', 'reviews_item_id', 'sentiment_analysis']]
df_RS.shape

(57397, 3)

El modelo que emplearemos es el "Descomposición en Valores Singulares" (SVD). Este es un algoritmo de factorización matricial que se utiliza para predecir calificaciones ⭐ o preferencias de los usuarios 👤 para ciertos elementos 🎮, basándose en calificaciones previas 📊.

In [7]:
# Se instancia la clase "Reader()" de Surprise:
reader = Reader(rating_scale = (0,2))

# Se carga la data en la clase respectiva:
data = Dataset.load_from_df(df_RS, reader)

# Se separan los datos en los grupos de entrenamiento y prueba:
trainset, testset = train_test_split(data, test_size = 0.20)

# Instanciamos el modelo:
model = SVD()

# Entrenamos:
model.fit(trainset)

# Se predice sobre el grupo de prueba
predictions = model.test(testset)

In [8]:
from surprise import accuracy

accuracy.rmse(predictions)

RMSE: 0.5832


0.5832134657273986

Dado que la escala de la columna de análisis de sentimiento varía de 0 a 2, un valor de RMSE cercano a 1 podría indicar cierta dificultad del modelo para evaluar correctamente los ítems.

Por este motivo, llevamos a cabo una exploración exhaustiva de los mejores hiperparámetros del modelo utilizando la técnica de búsqueda en grilla y validación cruzada. Nuestro objetivo es identificar aquellos hiperparámetros que minimicen el error al máximo posible. 🧐🔍✨

In [9]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)

# Se carga la data en la clase respectiva:
data = Dataset.load_from_df(df_RS, reader)

# Se entrena el modelo sobre el grupo de entrenamiento:
gs.fit(data)

print("Mejores hiperparámetros: "+str(gs.best_params['rmse']))
print("Mejor Score: "+str(gs.best_score['rmse'])+'\n')

Mejores hiperparámetros: {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.2}
Mejor Score: 0.5793409886717916



Finalmente, configuramos el modelo con los hiperparámetros óptimos que obtuvimos previamente. 🛠️🔧

In [10]:
# Instanciamos el modelo:
model1 = SVD(n_factors = 5, n_epochs = 20, lr_all = 0.002, reg_all = 0.2)

# Entrenamos:
model1.fit(trainset)

# Se predice sobre el grupo de prueba
predictions1 = model1.test(testset)

# Evaluamos:
accuracy.rmse(predictions1)

RMSE: 0.5755


0.5755277912261267

Guardamos el modelo entrenado en un archivo pkl:

In [11]:
# Ruta de la carpeta donde deseas guardar el archivo
folder_path = "../0 Dataset"

# Nombre del archivo
file_name = "RS_model.pkl"

# Ruta completa al archivo
file_path = os.path.join(folder_path, file_name)

# Guardar el modelo en la carpeta especificada
with open(file_path, 'wb') as file:
    pickle.dump(model1, file)

In [12]:
with open('RS_model.pkl', 'wb') as file:
    pickle.dump(model1, file)

## Endpoint API:

A continuación se crea la función que permitirá obtener, a partir de un id de usuario, una lista con 5 recomendaciones de juegos para el mismo:

In [13]:
def recomendacion_juego(user_id: str):
    '''
    Devuelve una lista con 5 sugerencias de juegos para el usuario seleccionado.
    Ejemplo de retorno: {'Sugerencias para el usuario 76561197970982479': ['1. RWBY: Grimm Eclipse', '2. Rogue Legacy', '3. Dust: An Elysian Tail', "4. King Arthur's Gold", '5. RIFT']}
    '''
    # Si el ID de usuario no se encuentra en los dataframes:
    if user_id not in df_user_reviews['user_id'].values:
        return f"ERROR: El ID de usuario {user_id} no existe en la base de datos."  # se imprime un mensaje de error
    else:
        # Se asigna el ID ingresado a la variable user
        user = user_id

        # En primer lugar, se extraen los juegos que el usuario ya ha jugado:
        df_rev_games = pd.merge(df_user_reviews, df_steam_games, left_on="reviews_item_id", right_on="id", how="inner")
        juegos_jugados = df_rev_games[df_rev_games['user_id'] == user]

        # Se eliminan del dataframe de juegos los jugados por el usuario
        df_user = df_steam_games[["id", "app_name"]].drop(juegos_jugados.id, errors='ignore')

        # Ruta completa al archivo RS_model.pkl
        ruta_modelo = '../0 Dataset/RS_model.pkl'

        # Se carga el modelo de Sistema de Recomendación entrenado desde el archivo especificado
        with open(ruta_modelo, 'rb') as file:
            RS_model = pickle.load(file)

        # Se realizan las predicciones y se agregan en una nueva columna:
        df_user['estimate_Score'] = df_user['id'].apply(lambda x: RS_model.predict(user, x).est)

        # Se ordena el dataframe de manera descendente en función al score y se seleccionan los 5 principales:
        sugerencias = df_user.sort_values('estimate_Score', ascending=False)["app_name"].head(5).to_list()

        # Se crea la llave del diccionario de retorno
        llave_dic = f'Sugerencias para el usuario {user}'

        # Se da formato a las 5 sugerencias:
        sugerencias_formateadas = [f'{i+1}. {sugerencia}' for i, sugerencia in enumerate(sugerencias)]

        # Se devuelven los resultados en un diccionario
        return {llave_dic: sugerencias_formateadas}


In [14]:
# Se prueba la funcion
recomendacion_juego("76561197970982479")

{'Sugerencias para el usuario 76561197970982479': ['1. Source Filmmaker',
  '2. Arma II: DayZ Mod',
  '3. Free to Play',
  '4. Guns of Icarus Online',
  '5. The Wolf Among Us']}